# Visual Analysis of the Steam Dataset

Please download the datasets at the following link and follow the instructions in `data/README.md`. Keep note of the pathnames used to load in the datasets and ensure they match what is in `data/` directory.

https://www.gigasheet.com/sample-data/steam-game-reviews

https://www.kaggle.com/datasets/mohamedtarek01234/steam-games-reviews-and-rankings

# A. Data Card

This section will explain what is contained in each point of our dataset.

### Datasets

*Steam Game Reviews:* https://www.gigasheet.com/sample-data/steam-game-reviews ~464MB

This dataset consists of ~990K rows of reviews taken from SteamDB, a database for the video game vendor, Steam.

*Steam Game Reviews and Rankings:* https://www.kaggle.com/datasets/mohamedtarek01234/steam-games-reviews-and-rankings ~1MB
 
This dataset is where we source game metadata to enrich our review information. This will help give better insights into the relationship between the kind of game and the reviews.

**Shape**

*Steam Game Reviews:* (992153, 8)

*Steam Game Reviews and Rankings:* (290, 13)

*Merged Working Dataset:* (992153, 24)

**Time**

This dataset covers review information from 2010-2024, with far more data coming from more recent time. (View visuals for review count by month/year)

### Column Dictionary

**For the working merged dataset**

**Provided Columns**    
`review` - The user's review string for the game   
`hours_played` - the amount of hours of the game the user played when they posted the review   
`helpful` - the amount of 'helpful' votes the user received from other users for their review   
`funny` - the amount of 'funny' votes the user received from other users for their review   
`recommendation` - whether the user recommends the game or not (0 - no, 1 - yes)   
`date` - the date when the user posted the review   
`game_name` - name of the game the user reviewed   
`username` - the user's personal alias   
`short_description` - the shortened description of the game from SteamDB   
`long_description` - the full description of the game from SteamDB   
`genres` - The list of genres that the game belongs to (ie. Action, Adventure, Horror...)   
`minimum_system_requirements` - the minimum hardware specs needed to run the game   
`recommended_system_requirements` - the hardware specs recommended by the developer to run the game smoothly   
`release_date` - the date when the game was released   
`developer` - the studio who developed the game   
`publisher` - the parent company that licensed the development of the game   
`overall_player_rating` - the overall average rating all reviews gave to the game (categorical)   
`number_of_reviews_from_purchased_people` - total number of reviews from players who purchased the game   
`number_of_english_reviews` - total number of reviews in english from players who purchased the game   
`link` - the link to the Steam page of the game   

**Created Columns by Us**     
`popular` - boolean value whether or not the review has >50 helpful votes   
`cat_playtime` - categorized bucketing for playtime hours     
`rec_ratio` - the ratio for recommended reviews to total reviews for the game based on the dataset reviews     
`in_sale` - a boolean that tells whether the review was posted during major sale periods (Summer - July, Winter - January)    

### Missingness Snapshot

*Steam Reviews*   
503 total missing reviews    
81 usernames    

*Steam Game Metadata*    
13 short descriptions (can be manually filled in)   

*Merged Working Dataset*    
Dates                                      1775    
short_description                          66892       
long_description                           66892    
genres                                     66892     
minimum_system_requirement                 66892   
recommend_system_requirement               66892    
release_date                               76879    
developer                                  66892    
publisher                                  66892    
overall_player_rating                      66892   
number_of_reviews_from_purchased_people    66892    
number_of_english_reviews                  66892   
link                                       66892   

The missing information from the merged dataset stems from the 11 games not covered by the metadata dataset. Can be filled manually.

### Quirks

1. Recent reviews do not include the year (because on Steam if a review is recent then it is just denoted by month and day). This was remedied by simply applying the year 2024 to all reviews missing a year. (data was collected in September of 2024)
2. A few usernames are poorly formatted and need to be fixed
3. 11 Games are missing metadata and can be filled manually
4. If a publisher owns their own development studio, then the publisher and developer will be the same.
5. A few of the review fields contain the year and need to be removed.
6. Genres, system requirements, and long_description are all list objects.

# 2 - Setup

### 2.1 Imports

Handles imports needed to run all cells in the notebook. If you haven't yet please run `pip install -r requirements.txt` 
in the main directory console to get necessary dependencies.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### 2.2 Load Datasets

This will load in all datasets that accompany this project. If you have not already, please follow the instructions in `data/README.md` to download the datasets required.

In [3]:
reviews_path = '../data/Steam Game Reviews export 2025-09-20 21-34-15.csv' # Replace with the path to your reviews CSV File
game_metadata_path = '../data/games_description.csv' # Replace with path to game metadata CSV file, NOTE make sure to go to parent dir in the path


steamreviews = pd.read_csv(reviews_path)
gamemetadata = pd.read_csv(game_metadata_path)
steamreviews.head()

,review,hours_played,helpful,funny,recommendation,date,game_name,username
0,The game itself is also super fun. The PvP and...,39.9,1152,13,Recommended,14 September,"Warhammer 40,000: Space Marine 2",Sentinowl\n224 products in account
1,Never cared much about Warhammer until this ga...,91.5,712,116,Recommended,13 September,"Warhammer 40,000: Space Marine 2",userpig\n248 products in account
2,A salute to all the fallen battle brothers who...,43.3,492,33,Recommended,14 September,"Warhammer 40,000: Space Marine 2",Imparat0r\n112 products in account
3,this game feels like it was made in the mid 20...,16.8,661,15,Recommended,14 September,"Warhammer 40,000: Space Marine 2",Fattest_falcon
4,Reminds me of something I've lost. A genuine g...,24.0,557,4,Recommended,12 September,"Warhammer 40,000: Space Marine 2",Jek\n410 products in account


In [4]:
gamemetadata.head()

,name,short_description,long_description,genres,minimum_system_requirement,recommend_system_requirement,release_date,developer,publisher,overall_player_rating,number_of_reviews_from_purchased_people,number_of_english_reviews,link
0,Black Myth: Wukong,Black Myth: Wukong is an action RPG rooted in ...,About This Game\n\t\t\t\t\t\t\tBlack Myth: Wuk...,"['Mythology', 'Action RPG', 'Action', 'RPG', '...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"19 Aug, 2024",['Game Science'],['Game Science'],Overwhelmingly Positive,"(654,820)","51,931",https://store.steampowered.com/app/2358720/Bla...
1,Counter-Strike 2,"For over two decades, Counter-Strike has offer...",About This Game\n\t\t\t\t\t\t\tFor over two de...,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...","['OS: Windows® 10', 'Processor: 4 hardware CPU...","['OS: Windows® 10', 'Processor: 4 hardware CPU...","21 Aug, 2012",['Valve'],['Valve'],Very Positive,"(8,313,603)","2,258,990",https://store.steampowered.com/app/730/Counter...
2,"Warhammer 40,000: Space Marine 2",Embody the superhuman skill and brutality of a...,About This Game\nEmbody the superhuman skill a...,"['Warhammer 40K', 'Action', 'Third-Person Shoo...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"9 Sep, 2024",['Saber Interactive'],['Focus Entertainment'],Very Positive,"(81% of 62,791) All Time","51,920",https://store.steampowered.com/app/2183900/War...
3,Cyberpunk 2077,"Cyberpunk 2077 is an open-world, action-advent...",About This Game\nCyberpunk 2077 is an open-wor...,"['Cyberpunk', 'Open World', 'Nudity', 'RPG', '...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"10 Dec, 2020",['CD PROJEKT RED'],['CD PROJEKT RED'],Very Positive,"(680,264)","324,124",https://store.steampowered.com/app/1091500/Cyb...
4,ELDEN RING,THE CRITICALLY ACCLAIMED FANTASY ACTION RPG. R...,About This Game\nTHE CRITICALLY ACCLAIMED FANT...,"['Souls-like', 'Dark Fantasy', 'Open World', '...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"25 Feb, 2022","['FromSoftware, Inc.']","['FromSoftware, Inc.', 'Bandai Namco Entertain...",Very Positive,"(705,261)","491,741",https://store.steampowered.com/app/1245620/ELD...


### 2.3 Merge Datasets

We can combine the review dataset and game metadata dataset to enrich each review.

In [5]:
steamdataset = pd.merge(steamreviews, gamemetadata, left_on='game_name', right_on='name', how='left') ## This is like a join in SQL, so we join based on the game name
steamdataset.drop(columns=['name'], inplace=True) # don't need 2 names :)
steamdataset

,review,hours_played,helpful,funny,recommendation,date,game_name,username,short_description,long_description,genres,minimum_system_requirement,recommend_system_requirement,release_date,developer,publisher,overall_player_rating,number_of_reviews_from_purchased_people,number_of_english_reviews,link
0,The game itself is also super fun. The PvP and...,39.9,1152,13,Recommended,14 September,"Warhammer 40,000: Space Marine 2",Sentinowl\n224 products in account,Embody the superhuman skill and brutality of a...,About This Game\nEmbody the superhuman skill a...,"['Warhammer 40K', 'Action', 'Third-Person Shoo...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"9 Sep, 2024",['Saber Interactive'],['Focus Entertainment'],Very Positive,"(81% of 62,791) All Time","51,920",https://store.steampowered.com/app/2183900/War...
1,Never cared much about Warhammer until this ga...,91.5,712,116,Recommended,13 September,"Warhammer 40,000: Space Marine 2",userpig\n248 products in account,Embody the superhuman skill and brutality of a...,About This Game\nEmbody the superhuman skill a...,"['Warhammer 40K', 'Action', 'Third-Person Shoo...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"9 Sep, 2024",['Saber Interactive'],['Focus Entertainment'],Very Positive,"(81% of 62,791) All Time","51,920",https://store.steampowered.com/app/2183900/War...
2,A salute to all the fallen battle brothers who...,43.3,492,33,Recommended,14 September,"Warhammer 40,000: Space Marine 2",Imparat0r\n112 products in account,Embody the superhuman skill and brutality of a...,About This Game\nEmbody the superhuman skill a...,"['Warhammer 40K', 'Action', 'Third-Person Shoo...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"9 Sep, 2024",['Saber Interactive'],['Focus Entertainment'],Very Positive,"(81% of 62,791) All Time","51,920",https://store.steampowered.com/app/2183900/War...
3,this game feels like it was made in the mid 20...,16.8,661,15,Recommended,14 September,"Warhammer 40,000: Space Marine 2",Fattest_falcon,Embody the superhuman skill and brutality of a...,About This Game\nEmbody the superhuman skill a...,"['Warhammer 40K', 'Action', 'Third-Person Shoo...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"9 Sep, 2024",['Saber Interactive'],['Focus Entertainment'],Very Positive,"(81% of 62,791) All Time","51,920",https://store.steampowered.com/app/2183900/War...
4,Reminds me of something I've lost. A genuine g...,24.0,557,4,Recommended,12 September,"Warhammer 40,000: Space Marine 2",Jek\n410 products in account,Embody the superhuman skill and brutality of a...,About This Game\nEmbody the superhuman skill a...,"['Warhammer 40K', 'Action', 'Third-Person Shoo...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"9 Sep, 2024",['Saber Interactive'],['Focus Entertainment'],Very Positive,"(81% of 62,791) All Time","51,920",https://store.steampowered.com/app/2183900/War...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992148,2022 Early Access Review Loads of fun!Initiall...,4.2,1,0,Recommended,"August 4, 2022",Turbo Golf Racing,"Fatal Exit\n1,902 products in account",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
992149,2022 Early Access Review Great fun!,8.5,1,0,Recommended,"August 4, 2022",Turbo Golf Racing,cleybaR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
992150,2022 Early Access Review Excellent pick up and...,83.3,2,0,Recommended,"August 4, 2022",Turbo Golf Racing,Sim\n600 products in account,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
992151,2022 Early Access Review This game is so much ...,3.8,1,0,Recommended,"August 4, 2022",Turbo Golf Racing,Fatboybadboy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Hypotheses

### Question 1
What correlation is there between positive reviews and player retention?

Our hypothesis for this is that there is a positive correlation. It makes sense to assume that a well received game will have longer player retention, but we can test this with a regression plot

In [6]:
# TODO work on plots